In [1]:
import pandas as pd
import sqlite3

In [2]:
bihar_election=pd.read_csv("Bihar_Election_Results.csv")

In [3]:
bihar_election.columns=['Election_Year','Position','Name','Votes','Percent_Votes','Party','AC_name','AC_no']
bihar_election;

In [4]:
for i, row_value in bihar_election['Percent_Votes'].iteritems():
    row_value = str(row_value)
    row_value = row_value.replace('%','')
    bihar_election.at[i,'Percent_Votes']= row_value

In [5]:
for columns in list(bihar_election.columns.values):
    bihar_election[columns] = pd.to_numeric(bihar_election[columns],errors='ignore')

In [6]:
con = sqlite3.connect('bihar.db')
c = con.cursor()
bihar_election.to_sql('bihar',con,if_exists='append', index=False)
cursorObj = con.cursor()

In [7]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

In [8]:
#printmd("**('Election_Year','Position','Name','Votes','Percent_Votes','Party','AC_name','AC_no')**")
# def scriptexecution(filename):
 #   with open(filename, 'r') as s:
  #      sql_script = s.read()
   #     cursorObj.execute(sql_script)
    #s.closed
 
#scriptexecution('a.sql')
#A
cursorObj.execute('CREATE TABLE my AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('SELECT election_year,party,votes FROM my AS t where votes = (SELECT max(votes) FROM my WHERE t.election_year=election_year);')
rows = cursorObj.fetchall()
printmd("**('Election_Year ',' Party ',' Votes')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE my;')

**('Election_Year ',' Party ',' Votes')**

('2015', 'Bharatiya Janta Party', 9308015)
('2010', 'Janata Dal (United)', 6561906)
('2005-Feb', 'Rashtriya Janata Dal', 6140223)
('2005-Oct', 'Rashtriya Janata Dal', 5525081)


In [9]:
#B
cursorObj.execute('SELECT ac_name FROM( SELECT ac_name,COUNT(DISTINCT party) cnt FROM( SELECT election_year,ac_name,party FROM bihar where position=1) GROUP BY ac_name) WHERE cnt=4;')

rows = cursorObj.fetchall()
printmd("**('Election_Year ',' Party ')**")
for row in rows:
    print(row)

**('Election_Year ',' Party ')**

('Amour',)
('Bachhwara',)
('Balrampur',)
('Belsand',)
('Chakai',)
('Jhajha',)
('Kadwa',)
('Lauriya',)
('Madhubani',)
('Pipra',)
('Ramgarh',)
('Tarari',)
('Thakurganj',)


In [10]:
#C
cursorObj.execute('CREATE TABLE a1 AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('CREATE TABLE one AS SELect election_year,party,votes FROM a1 WHERE election_year="2015";')
cursorObj.execute('CREATE TABLE two AS SELect election_year,party,votes FROM a1 WHERE election_year="2010";')
cursorObj.execute('CREATE TABLE three AS SELect election_year,party,votes FROM a1 WHERE election_year="2005-Feb";')
cursorObj.execute('CREATE TABLE four AS SELect election_year,party,votes FROM a1 WHERE election_year="2005-Oct";')
cursorObj.execute('CREATE TABLE onet AS SElECT election_year,party FROM one ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('CREATE TABLE twot AS SElECT election_year,party FROM two ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('CREATE TABLE threet AS SElECT election_year,party FROM three ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('CREATE TABLE fourt AS SElECT election_year,party FROM four ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('INSERT INTO twot SELECT * FROM onet;')
cursorObj.execute('INSERT INTO twot SELECT * FROM threet;')
cursorObj.execute('INSERT INTO twot SELECT * FROM fourt;')
cursorObj.execute('SELECT * FROM twot;')
rows = cursorObj.fetchall()
printmd("**('Election_Year ',' Party ')**")
for row in rows:
    print(row)
    
cursorObj.execute("DROP TABLE a1;")
cursorObj.execute('DROP TABLE one;')
cursorObj.execute('DROP TABLE two;')
cursorObj.execute('DROP TABLE three;')
cursorObj.execute('DROP TABLE four;')
cursorObj.execute('DROP TABLE onet;')
cursorObj.execute('DROP TABLE twot;')
cursorObj.execute('DROP TABLE threet;')
cursorObj.execute('DROP TABLE fourt;')

**('Election_Year ',' Party ')**

('2010', 'Janata Dal (United)')
('2010', 'Rashtriya Janata Dal')
('2010', 'Bharatiya Janta Party')
('2010', 'Indian National Congress')
('2010', 'Lok Jan Shakti Party')
('2010', 'Bahujan Samaj Party')
('2010', 'Nationalist Congress Party')
('2010', 'Communist Party Of India (MARXIST-LENINIST) (LIBERATION)')
('2015', 'Bharatiya Janta Party')
('2015', 'Rashtriya Janata Dal')
('2015', 'Janata Dal (United)')
('2015', 'Indian National Congress')
('2015', 'Lok Jan Shakti Party')
('2015', 'Rashtriya Lok Samta Party')
('2015', 'Hindustani Awam Morcha (Secular)')
('2015', 'Bahujan Samaj Party')
('2005-Feb', 'Rashtriya Janata Dal')
('2005-Feb', 'Janata Dal (United)')
('2005-Feb', 'Lok Jan Shakti Party')
('2005-Feb', 'Bharatiya Janta Party')
('2005-Feb', 'Indian National Congress')
('2005-Feb', 'Bahujan Samaj Party')
('2005-Feb', 'Samajwadi Party')
('2005-Feb', 'Communist Party Of India (MARXIST-LENINIST) (LIBERATION)')
('2005-Oct', 'Rashtriya Janata Dal')
('2005-Oct', 'Janata Dal (United)')
('20

In [11]:
#D    
cursorObj.execute('CREATE TABLE a1 AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('CREATE TABLE one AS SELect election_year,party,votes FROM a1 WHERE election_year="2015";')
cursorObj.execute('CREATE TABLE onet AS SELECT party FROM one ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('SELECT party,ac_name,min(votes) min_votes FROM ( SELECT election_year,party,ac_name,votes FROM bihar WHERE election_year!="2015" AND party IN (SELECT party from onet) ) GROUP BY party,ac_name;')
rows = cursorObj.fetchall()
printmd("**(' Party ',' AC_name' ,' Votes')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE a1;')
cursorObj.execute('DROP TABLE one;')
cursorObj.execute('DROP TABLE onet;')

**(' Party ',' AC_name' ,' Votes')**

('Bahujan Samaj Party', 'Agiaon', 511)
('Bahujan Samaj Party', 'Agiaon (SC)', 2693)
('Bahujan Samaj Party', 'Alamnagar', 1310)
('Bahujan Samaj Party', 'Alauli', 2702)
('Bahujan Samaj Party', 'Alauli (SC)', 5365)
('Bahujan Samaj Party', 'Alinagar', 1896)
('Bahujan Samaj Party', 'Amarpur', 2425)
('Bahujan Samaj Party', 'Amnour', 1702)
('Bahujan Samaj Party', 'Amour', 2227)
('Bahujan Samaj Party', 'Araria', 1374)
('Bahujan Samaj Party', 'Arrah', 1041)
('Bahujan Samaj Party', 'Arwal', 3870)
('Bahujan Samaj Party', 'Asthawan', 979)
('Bahujan Samaj Party', 'Atri', 2639)
('Bahujan Samaj Party', 'Aurai', 860)
('Bahujan Samaj Party', 'Aurangabad', 3734)
('Bahujan Samaj Party', 'Babubarhi', 2061)
('Bahujan Samaj Party', 'Bachhwara', 945)
('Bahujan Samaj Party', 'Bagaha', 3015)
('Bahujan Samaj Party', 'Bahadurganj', 4232)
('Bahujan Samaj Party', 'Bahadurpur', 1179)
('Bahujan Samaj Party', 'Baikunthpur', 1012)
('Bahujan Samaj Party', 'Baisi', 1255)
('Bahujan Samaj Party', 'Bajpatti', 2109)
('Bahuj

In [12]:
#E    
cursorObj.execute('CREATE TABLE temm as SELECT election_year,ac_name,COUNT(party) cnt FROM bihar WHERE percent_votes>10.00 GROUP BY election_year,ac_name;')
cursorObj.execute('SELECT ac_name from temm where cnt>=3 group by ac_name having COUNT(cnt) >=4;')
rows = cursorObj.fetchall()
printmd("**(' AC_name')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE temm;')

**(' AC_name')**

('Amnour',)
('Arwal',)
('Baisi',)
('Balrampur',)
('Banka',)
('Chainpur',)
('Dehri',)
('Harlakhi',)
('Jagdishpur',)
('Kadwa',)
('Kanti',)
('Karakat',)
('Kargahar',)
('Katihar',)
('Lauriya',)
('Nabinagar',)
('Narkatiaganj',)
('Pipra',)
('Pranpur',)
('Ramgarh',)
('Sheikhpura',)
('Sheohar',)
('Tarari',)
('Ujiarpur',)
('Valmiki Nagar',)


In [13]:
#F   
cursorObj.execute('CREATE TABLE cur AS SELECT election_year,ac_name,party FROM bihar WHERE position=1;')
cursorObj.execute('SELECT ac_name FROM(SELECT ac_name,COUNT(DISTINCT party) cnt2 FROM cur GROUP BY ac_name) cur2 WHERE cur2.cnt2=1;')
rows = cursorObj.fetchall()
printmd("**(' AC_name')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE cur;')

**(' AC_name')**

('Agiaon (SC)',)
('Alauli (SC)',)
('Bakhri (SC)',)
('Banmankhi (SC)',)
('Barachatti (SC)',)
('Bathnaha (SC)',)
('Belhar',)
('Bettiah',)
('Bhorey (SC)',)
('Bibhutpur',)
('Bochaha (SC)',)
('Bodh Gaya (SC)',)
('Chanpatia',)
('Chenari (SC)',)
('Darauli (SC)',)
('Dhauraiya (SC)',)
('Dhuraiya',)
('Fatwah',)
('Garkha (SC)',)
('Gopalpur',)
('Gora Bauram',)
('Goriakothi',)
('Harsidhi (SC)',)
('Imamganj (SC)',)
('Kalyanpur (SC)',)
('Katoria (ST)',)
('Khagaria',)
('Korha (SC)',)
('Kumhrar',)
('Kumhrarh',)
('Kurtha',)
('Kusheshwar Asthan (SC)',)
('Kutumba (SC)',)
('Maharajganj',)
('Mahnar',)
('Makhadumapur (SC)',)
('Manihari (ST)',)
('Masaurhi (SC)',)
('Mohania (SC)',)
('Nalanda',)
('Nirmali',)
('Parihar',)
('Patepur (SC)',)
('Phulwari (SC)',)
('Pirpainti (SC)',)
('Raja Pakar (SC)',)
('Rajauli (SC)',)
('Rajgir (SC)',)
('Rajnagar (SC)',)
('Rajpur',)
('Rajpur (SC)',)
('Ramnagar (SC)',)
('Raniganj (SC)',)
('Raxaul',)
('Rosera (SC)',)
('Runnisaidpur',)
('Sakra (SC)',)
('Sikandra (SC)',)
('Singheshwar'

In [14]:
#G
cursorObj.execute('CREATE TABLE var AS SELECT election_year,ac_name,percent_votes FROM bihar WHERE position=1;')
cursorObj.execute('CREATE TABLE var2 AS SELECT election_year,ac_name,percent_votes FROM bihar WHERE position=2;')
cursorObj.execute('SELECT var3.ac_name FROM(SELECT var.election_year,var.ac_name, abs(var.percent_votes - var2.percent_votes) diff FROM var,var2 WHERE var.election_year=var2.election_year AND var.ac_name = var2.ac_name) var3 where var3.diff<10.00 ;')

rows = cursorObj.fetchall()
printmd("**(' AC_name')**")
for row in rows:
    print(row)
    
cursorObj.execute('DROP TABLE var;')
cursorObj.execute('DROP TABLE var2;')

**(' AC_name')**

('Lauriya',)
('Sikta',)
('Raxaul',)
('Narkatia',)
('Govindganj',)
('Pipra',)
('Pipra',)
('Madhuban',)
('Dhaka',)
('Sheohar',)
('Parihar',)
('Sursand',)
('Bajpatti',)
('Sitamarhi',)
('Runisaidpur',)
('Harlakhi',)
('Khajauli',)
('Babubarhi',)
('Bisfi',)
('Madhubani',)
('Rajnagar',)
('Phulparas',)
('Narpatganj',)
('Sikti',)
('Bahadurganj',)
('Thakurganj',)
('Kishanganj',)
('Kochadhaman',)
('Baisi',)
('Kasba',)
('Balrampur',)
('Pranpur',)
('Manihari',)
('Madhepura',)
('Saharsa',)
('Mahishi',)
('Kusheshwar Asthan',)
('Alinagar',)
('Darbhanga Rural',)
('Hayaghat',)
('Bahadurpur',)
('Keoti',)
('Minapur',)
('Kurhani',)
('Kanti',)
('Sahebganj',)
('Barauli',)
('Ziradei',)
('Darauli',)
('Manjhi',)
('Baniapur',)
('Taraiya',)
('Marhaura',)
('Garkha',)
('Parsa',)
('Vaishali',)
('Raja Pakar',)
('Raghopur',)
('Manhar',)
('Samastipur',)
('Ujiarpur',)
('Morwa',)
('Rosera',)
('Hasanpur',)
('Cheria Bariarpur',)
('Bachhwara',)
('Teghra',)
('Parbatta',)
('Bihpur',)
('Pirpainti',)
('Kahalgaon',)
('Bhagalpur'

In [15]:
#H
cursorObj.execute('CREATE TABLE a1 AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('CREATE TABLE one AS SELect election_year,party,votes FROM a1 WHERE election_year="2005-Feb";')
cursorObj.execute('CREATE TABLE onet AS SELECT party FROM one ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('CREATE TABLE first AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2005-Feb" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE second AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2005-Oct" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE third AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2010" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE fourth AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2015" AND party in ( SELECT party FROM onet); ')
cursorObj.execute('CREATE TABLE second2 AS SELECT second.ac_name,second.party,second.votes FROM first,second WHERE first.ac_name=second.ac_name AND first.party = second.party AND first.votes<second.votes ;')
cursorObj.execute('CREATE TABLE third2 AS SELECT third.ac_name,third.party,third.votes FROM second2,third WHERE second2.ac_name=third.ac_name AND second2.party = third.party AND second2.votes<third.votes ;')
cursorObj.execute('SELECT fourth.party,fourth.ac_name FROM third2,fourth WHERE third2.ac_name=fourth.ac_name AND third2.party = fourth.party AND third2.votes<fourth.votes ;')

rows = cursorObj.fetchall()
printmd("**(' Party ',' AC_name')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE a1;')
cursorObj.execute('DROP TABLE one;')
cursorObj.execute('DROP TABLE onet;')
cursorObj.execute('DROP TABLE first;')
cursorObj.execute('DROP TABLE second;')
cursorObj.execute('DROP TABLE third;')
cursorObj.execute('DROP TABLE fourth;')
cursorObj.execute('DROP TABLE second2;')
cursorObj.execute('DROP TABLE third2;')

**(' Party ',' AC_name')**

('Janata Dal (United)', 'Nautan')
('Bharatiya Janta Party', 'Chanpatia')
('Janata Dal (United)', 'Sikta')
('Bharatiya Janta Party', 'Raxaul')
('Bharatiya Janta Party', 'Sugauli')
('Bahujan Samaj Party', 'Govindganj')
('Janata Dal (United)', 'Kalyanpur')
('Janata Dal (United)', 'Kalyanpur')
('Lok Jan Shakti Party', 'Kalyanpur')
('Janata Dal (United)', 'Madhuban')
('Bahujan Samaj Party', 'Madhuban')
('Indian National Congress', 'Riga')
('Bharatiya Janta Party', 'Sitamarhi')
('Rashtriya Janata Dal', 'Khajauli')
('Janata Dal (United)', 'Babubarhi')
('Rashtriya Janata Dal', 'Jhanjharpur')
('Janata Dal (United)', 'Phulparas')
('Janata Dal (United)', 'Nirmali')
('Janata Dal (United)', 'Supaul')
('Rashtriya Janata Dal', 'Chhatapur')
('Janata Dal (United)', 'Jokihat')
('Janata Dal (United)', 'Dhamdaha')
('Indian National Congress', 'Purnia')
('Rashtriya Janata Dal', 'Saharsa')
('Bharatiya Janta Party', 'Saharsa')
('Rashtriya Janata Dal', 'Alinagar')
('Rashtriya Janata Dal', 'Darbhanga Rural')
(

In [16]:
#I
cursorObj.execute('CREATE TABLE a1 AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('CREATE TABLE one AS SELect election_year,party,votes FROM a1 WHERE election_year="2005-Feb";')
cursorObj.execute('CREATE TABLE onet AS SELECT party FROM one ORDER BY votes DESC LIMIT 8;')
cursorObj.execute('CREATE TABLE first AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2005-Feb" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE second AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2005-Oct" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE third AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2010" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE fourth AS SELECT ac_name,party,votes FROM bihar WHERE election_year = "2015" AND party in ( SELECT party FROM onet);')
cursorObj.execute('CREATE TABLE second2 AS SELECT second.ac_name,second.party,second.votes FROM first,second WHERE first.ac_name=second.ac_name AND first.party = second.party AND first.votes>second.votes ; ')
cursorObj.execute('CREATE TABLE third2 AS SELECT third.ac_name,third.party,third.votes FROM second2,third WHERE second2.ac_name=third.ac_name AND second2.party = third.party AND second2.votes>third.votes ; ')
cursorObj.execute('SELECT fourth.party,fourth.ac_name FROM third2,fourth WHERE third2.ac_name=fourth.ac_name AND third2.party = fourth.party AND third2.votes>fourth.votes ;')

rows = cursorObj.fetchall()
printmd("**(' Party ',' AC_name')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE a1;')
cursorObj.execute('DROP TABLE one;')
cursorObj.execute('DROP TABLE onet;')
cursorObj.execute('DROP TABLE first;')
cursorObj.execute('DROP TABLE second;')
cursorObj.execute('DROP TABLE third;')
cursorObj.execute('DROP TABLE fourth;')
cursorObj.execute('DROP TABLE second2;')
cursorObj.execute('DROP TABLE third2;')


**(' Party ',' AC_name')**

('Bahujan Samaj Party', 'Narkatiaganj')
('Samajwadi Party', 'Madhuban')
('Bahujan Samaj Party', 'Dhaka')
('Bahujan Samaj Party', 'Jokihat')
('Bahujan Samaj Party', 'Kochadhaman')
('Bahujan Samaj Party', 'Barari')
('Bahujan Samaj Party', 'Bihariganj')
('Bahujan Samaj Party', 'Bahadurpur')
('Bahujan Samaj Party', 'Sahebganj')
('Samajwadi Party', 'Lalganj')
('Bahujan Samaj Party', 'Raghopur')
('Bahujan Samaj Party', 'Ujiarpur')
('Bahujan Samaj Party', 'Begusarai')
('Bahujan Samaj Party', 'Sultanganj')
('Bahujan Samaj Party', 'Nathnagar')
('Bahujan Samaj Party', 'Tarapur')
('Bahujan Samaj Party', 'Digha')
('Bahujan Samaj Party', 'Arrah')
('Bahujan Samaj Party', 'Dinara')
('Bahujan Samaj Party', 'Nokha')
('Bahujan Samaj Party', 'Karakat')
('Bahujan Samaj Party', 'Jahanabad')
('Bahujan Samaj Party', 'Gaya Town')
('Bahujan Samaj Party', 'Wazirganj')
('Bahujan Samaj Party', 'Hisua')


In [17]:
#J
cursorObj.execute('CREATE TABLE my AS SELECT election_year,party,sum(votes) votes FROM bihar where party!="Independent" GROUP BY party,election_year;')
cursorObj.execute('CREATE TABLE state_winners as SELECT election_year,party FROM my AS t where votes = (SELECT max(votes) FROM my WHERE t.election_year=election_year);')
cursorObj.execute('CREATE TABLE ac_winners AS SELECT election_year,ac_name,party FROM bihar WHERE position=1;')
cursorObj.execute('CREATE table tot as SELECT ac_winners.election_year,ac_winners.ac_name FROM ac_winners,state_winners WHERE ac_winners.election_year=state_winners.election_year AND ac_winners.party=state_winners.party;')
cursorObj.execute('SELECT ac_name FROM tot GROUP BY ac_name having count(DISTINCT election_year)=4; ')

rows = cursorObj.fetchall()
printmd("**('AC_name ')**")
for row in rows:
    print(row)

cursorObj.execute('DROP TABLE my;')
cursorObj.execute('DROP TABLE state_winners;')
cursorObj.execute('DROP TABLE ac_winners;')
cursorObj.execute('DROP TABLE tot;')

**('AC_name ')**

('Bihariganj',)
